In [1]:
import pandas as pd
import numpy as np
import re
import json
from pandas import json_normalize

### Process Prediction data

In [2]:
# Load Data
file_path = '/home/zhangwei/SFTChatGPT_for_chemtext_mining_/paragraph2MOF/gpt_results/prediction_of_paragraph2MOF_epoch_3_test_329.csv'
file_path = 'results/predictions/prediction_of_saved_models-merged_models-saved_models-llama2_13b_chat_qlora-train_329_lora_r8_lr0.0001-checkpoint-660.csv'
file_path = 'results/predictions/prediction_of_saved_models-Meta-Llama-3-8B-Instruct-train_329_lr5e-06_bs2-checkpoint-330.csv'  # 330 best
file_path = 'results/predictions/prediction_of_saved_models-Mistral-7B-Instruct-v0.2-train_329_lr5e-06_bs2-checkpoint-990.csv'  # 990 best
file_path = 'results/predictions/gpt_4_zero_shot_prediction_of_paragraph2MOF_test_329.csv'

df = pd.read_csv(file_path)
df['Generated Text'] = df['Generated Text'].str.replace('\n', '')
df

,Paragraph,Actual Text,Generated Text
0,"nMTV-MOF-5-ABCDEF, Zn4O(BDC)0.88(NH2-BDC)0.01(...","[{""compound name"": ""nMTV-MOF-5-ABCDEF, Zn4O(BD...","[{""compound name"": ""nMTV-MOF-5-ABCDEF"", ""metal..."
1,"nCo-MOF-74 Co2(DOT) (DOT=2,5-dihydroxyl-1,4-be...","[{""compound name"": ""nCo-MOF-74 Co2(DOT) (DOT=2...","[{""compound name"": ""nCo-MOF-74 Co2(DOT)"", ""met..."
2,nNi-MOF-74 Ni2(DOT): 5 mL of DOTH2 stock solut...,"[{""compound name"": ""nNi-MOF-74 Ni2(DOT)"", ""met...","[{""compound name"": ""nNi-MOF-74 Ni2(DOT)"", ""met..."
3,"nM3M-MOF-74, Co0.82Mg0.424Ni0.76(DOT): 5 mL of...","[{""compound name"": ""nM3M-MOF-74, Co0.82Mg0.424...","[{""compound name"": ""nM3M-MOF-74"", ""metal sourc..."
4,"nM5M-MOF-74, Co0.54Mg0.27Ni0.60Zn0.34Mn0.35(DO...","[{""compound name"": ""nM5M-MOF-74, Co0.54Mg0.27N...","[{""compound name"": ""nM5M-MOF-74, Co0.54Mg0.27N..."
...,...,...,...
324,"Synthesis of PPMOF 1 H6L (142 mg, 0.1 mmol) an...","[{""compound name"": ""PPMOF 1"", ""metal source"": ...","[{""compound name"": ""PPMOF 1"", ""metal source"": ..."
325,Synthesis of [Co(atz)(L1)0.5]·DMF (1). A mixtu...,"[{""compound name"": ""[Co(atz)(L1)0.5]·DMF"", ""me...","[{""compound name"": ""[Co(atz)(L1)0.5]·DMF"", ""me..."
326,Synthesis of [Co(atz)(L1)0.5]·DMF (1). A mixtu...,"[{""compound name"": ""[Co(atz)(L2)0.5]·DMF"", ""me...","[{""compound name"": ""[Co(atz)(L1)0.5]·DMF"", ""me..."
327,Synthesis of MOF Materials. [emi][Mn(btc)] (1)...,"[{""compound name"": ""[emi][Mn(btc)]"", ""metal so...","[{""compound name"": ""[emi][Mn(btc)]"", ""metal so..."


In [3]:
# GroundTruth: Extract 1-8 dictionaries of column 'Actual Text' , save into 8 .csv files

data_groundtruth_MOF = list(df['Actual Text'])

for i in range(8):
    dataframes = []
    for row in data_groundtruth_MOF:
        try:
            data_groundtruth_MOF_row = json.loads(row)
            data_groundtruth_MOF_row = json_normalize(data_groundtruth_MOF_row).iloc[i]
            dataframes.append(data_groundtruth_MOF_row)
        except Exception as e:
            dataframes.append({"compound name":"", "metal source":"", "metal amount":"", "linker":"", "linker amount":"", "modulator":"", "modulator amount or volume":"", "solvent" :"", "solvent volume" :"", "reaction temperature" :"", "reaction time":""})  

    dataframes_df = pd.DataFrame(dataframes).reset_index(drop=True)
    dataframes_df.to_csv(f'results/predictions/ground_truth_column{i+1}.csv', index = False)

In [4]:
# Prediction: Extract 1-8 dictionaries of column 'Generated Text' , save into 8 .csv files

default_row = pd.Series({
    'compound name': '',
    'metal source': '',
    'metal amount': '',
    'linker': '',
    'linker amount': '',
    'modulator': '',
    'modulator amount or volume': '',
    'solvent': '',
    'solvent volume': '',
    'reaction temperature': '',
    'reaction time': ''
})
print(type(default_row))

data_prediction_MOF = list(df['Generated Text'])

for i in range(8):
    dataframes = []
    for a,row in enumerate(data_prediction_MOF):
        try:
            data_prediction_MOF_row = json.loads(row)
            data_prediction_MOF_row = json_normalize(data_prediction_MOF_row).iloc[i]
            dataframes.append(data_prediction_MOF_row)
        except Exception as e:
            print(f"An error occurred: {e}")
            dataframes.append(default_row)       
    dataframes_df = pd.DataFrame(dataframes).reset_index(drop=True)
    dataframes_df.to_csv(f'results/predictions/prediction_column{i+1}.csv', index = False)

<class 'pandas.core.series.Series'>
An error occurred: Expecting value: line 1 column 358 (char 357)
An error occurred: Expecting value: line 1 column 329 (char 328)
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single positional indexer is out-of-bounds
An error occurred: single po

In [5]:
#ground_truth
df_ground_column1 = pd.read_csv('results/predictions/ground_truth_column1.csv')
df_ground_column2 = pd.read_csv('results/predictions/ground_truth_column2.csv')
df_ground_column3 = pd.read_csv('results/predictions/ground_truth_column3.csv')
df_ground_column4 = pd.read_csv('results/predictions/ground_truth_column4.csv')
df_ground_column5 = pd.read_csv('results/predictions/ground_truth_column5.csv')
df_ground_column6 = pd.read_csv('results/predictions/ground_truth_column6.csv')
df_ground_column7 = pd.read_csv('results/predictions/ground_truth_column7.csv')
df_ground_column8 = pd.read_csv('results/predictions/ground_truth_column8.csv')

# filter null
non_empty_rows_column1 = df_ground_column1.isnull().all(axis=1)
non_empty_rows_column2 = df_ground_column2.isnull().all(axis=1)
non_empty_rows_column3 = df_ground_column3.isnull().all(axis=1)
non_empty_rows_column4 = df_ground_column4.isnull().all(axis=1)
non_empty_rows_column5 = df_ground_column5.isnull().all(axis=1)
non_empty_rows_column6 = df_ground_column6.isnull().all(axis=1)
non_empty_rows_column7 = df_ground_column7.isnull().all(axis=1)
non_empty_rows_column8 = df_ground_column8.isnull().all(axis=1)

########################################################################################################################################
#多反应的数据,在第二个字典中不为NULL的，需要第一个到第8个字典的值,第一个字典只要第二个字典非NULL的，第1个之后的字典不全为空的字典都要
########################################################################################################################################
df_ground_column1_multiple1 = df_ground_column1[~non_empty_rows_column2]
df_ground_column1_multiple2 = df_ground_column2[~non_empty_rows_column2]
df_ground_column1_multiple3 = df_ground_column3[~non_empty_rows_column3]
df_ground_column1_multiple4 = df_ground_column4[~non_empty_rows_column4]
df_ground_column1_multiple5 = df_ground_column5[~non_empty_rows_column5]
df_ground_column1_multiple6 = df_ground_column6[~non_empty_rows_column6]
df_ground_column1_multiple7 = df_ground_column7[~non_empty_rows_column7]
df_ground_column1_multiple8 = df_ground_column8[~non_empty_rows_column8]

df_ground_columns_multiple = pd.concat([df_ground_column1_multiple1, df_ground_column1_multiple2, df_ground_column1_multiple3, df_ground_column1_multiple4,
                                         df_ground_column1_multiple5, df_ground_column1_multiple6, df_ground_column1_multiple7, df_ground_column1_multiple8])
df_ground_columns_multiple = df_ground_columns_multiple.fillna('N/A')
df_ground_columns_multiple


,compound name,metal source,metal amount,linker,linker amount,modulator,modulator amount or volume,solvent,solvent volume,reaction temperature,reaction time
17,PCN-56,ZrCl4,24 mg,TPDC derivative dicarboxylic acid ligand,30 or 56 mg,benzoic acid,240 mg,DMF,3.0 mL,100 °C or 120 °C,48 h or 16 h
51,[Zr6O6(OH)2(C10H2O4S3)4(C6H5COO)2(C3H7NO)6](C3...,ZrCl4,"223 mg, 1 mmol",H2dttdc,"190 mg, 0.67 mmol",benzoic acid,8 g,DMF,50 ml,120 °C,72 hours
52,DUT-52(Zr),ZrCl4,"230 mg, 1.03 mmol","H2(2,6-ndc)","216 mg, 1 mmol",acetic acid,"3 ml, 47.6 mmol",DMF,20 ml,120 °C,24 hours
55,ZrSQU,ZrCl4,"56 mg, 0.24 mmol",SQA,"82 mg, 0.72 mmol",HCl | acetic acid (AA),"12 mmol, 1 mL HCl 37% | 4 mL, 70 mmol","N,N dimethylformamide (DMF)","5 mL, 64 mmol",383 K,2-3 h
60,MIL-140B or ZrO[O2C-C10H6-CO2],ZrCl4,"699mg, 3mmol","Naphthalene-2,6-dicarboxylic acid (NDC) or Bip...","1300 mg, 6 mmol or 2.42 mg, 10 mmol",acetic acid (CH3CO2H),"342 μL, 359 mg, 6 mmol or 1430 μL, 1.50 g, 25 ...",DMF,"15 mL, 14.3 g, 195 mmol or 25 mL, 23.7 g, 324 ...",220 °C,6 hours
...,...,...,...,...,...,...,...,...,...,...,...
322,Complex 5,Zn(ClO4)2,0.5 mmol,L,"0.18 g, 0.5 mmol",N/A,N/A,"ethanol/methanol | N,N-dimethylformamide (DMF)","10 cm3, ratio in volume2:1",room temperature,0.5 h
17,PCN-57,ZrCl4,24 mg,TPDC derivative dicarboxylic acid ligand,30 or 56 mg,N/A,N/A,DMF,3.0 mL,100 °C or 120 °C,48 h or 16 h
322,Complex 6,Co(ClO4)2,0.5 mmol,L,"0.18 g, 0.5 mmol",N/A,N/A,"ethanol/methanol | N,N-dimethylformamide (DMF)","10 cm3, ratio in volume2:1",room temperature,0.5 h
17,PCN-58,ZrCl4,24 mg,TPDC derivative dicarboxylic acid ligand,30 or 56 mg,N/A,N/A,DMF,3.0 mL,100 °C or 120 °C,48 h or 16 h


### LLMs

In [ ]:
# Predictions
df_predict_column1 = pd.read_csv('results/predictions/prediction_column1.csv')
df_predict_column2 = pd.read_csv('results/predictions/prediction_column2.csv')
df_predict_column3 = pd.read_csv('results/predictions/prediction_column3.csv')
df_predict_column4 = pd.read_csv('results/predictions/prediction_column4.csv')
df_predict_column5 = pd.read_csv('results/predictions/prediction_column5.csv')
df_predict_column6 = pd.read_csv('results/predictions/prediction_column6.csv')
df_predict_column7 = pd.read_csv('results/predictions/prediction_column7.csv')
df_predict_column8 = pd.read_csv('results/predictions/prediction_column8.csv')

# columns
df_predict_column1 = df_predict_column1[['compound name', 'metal source', 'metal amount', 'linker', 'linker amount', 'modulator', 'modulator amount or volume', 'solvent', 'solvent volume', 'reaction temperature', 'reaction time']]
df_predict_column2 = df_predict_column2[['compound name', 'metal source', 'metal amount', 'linker', 'linker amount', 'modulator', 'modulator amount or volume', 'solvent', 'solvent volume', 'reaction temperature', 'reaction time']]
df_predict_column3 = df_predict_column3[['compound name', 'metal source', 'metal amount', 'linker', 'linker amount', 'modulator', 'modulator amount or volume', 'solvent', 'solvent volume', 'reaction temperature', 'reaction time']]
df_predict_column4 = df_predict_column4[['compound name', 'metal source', 'metal amount', 'linker', 'linker amount', 'modulator', 'modulator amount or volume', 'solvent', 'solvent volume', 'reaction temperature', 'reaction time']]
df_predict_column5 = df_predict_column5[['compound name', 'metal source', 'metal amount', 'linker', 'linker amount', 'modulator', 'modulator amount or volume', 'solvent', 'solvent volume', 'reaction temperature', 'reaction time']]
df_predict_column6 = df_predict_column6[['compound name', 'metal source', 'metal amount', 'linker', 'linker amount', 'modulator', 'modulator amount or volume', 'solvent', 'solvent volume', 'reaction temperature', 'reaction time']]
df_predict_column7 = df_predict_column7[['compound name', 'metal source', 'metal amount', 'linker', 'linker amount', 'modulator', 'modulator amount or volume', 'solvent', 'solvent volume', 'reaction temperature', 'reaction time']]
df_predict_column8 = df_predict_column8[['compound name', 'metal source', 'metal amount', 'linker', 'linker amount', 'modulator', 'modulator amount or volume', 'solvent', 'solvent volume', 'reaction temperature', 'reaction time']]

########################################################################################################################################
# #多反应的数据,在第二个字典中不为NULL的，需要第一个到第8个字典的值,第一个字典只要第二个字典非NULL的，第1个之后的字典不全为空的字典都要
########################################################################################################################################
df_predict_column1_multiple1 = df_predict_column1[~non_empty_rows_column2]
df_predict_column1_multiple2 = df_predict_column2[~non_empty_rows_column2]
df_predict_column1_multiple3 = df_predict_column3[~non_empty_rows_column3]
df_predict_column1_multiple4 = df_predict_column4[~non_empty_rows_column4]
df_predict_column1_multiple5 = df_predict_column5[~non_empty_rows_column5]
df_predict_column1_multiple6 = df_predict_column6[~non_empty_rows_column6]
df_predict_column1_multiple7 = df_predict_column7[~non_empty_rows_column7]
df_predict_column1_multiple8 = df_predict_column8[~non_empty_rows_column8]

df_predict_columns_multiple = pd.concat([df_predict_column1_multiple1, df_predict_column1_multiple2, df_predict_column1_multiple3, df_predict_column1_multiple4,
                                         df_predict_column1_multiple5, df_predict_column1_multiple6, df_predict_column1_multiple7, df_predict_column1_multiple8])
df_predict_columns_multiple = df_predict_columns_multiple.fillna('N/A')
df_predict_columns_multiple


,compound name,metal source,metal amount,linker,linker amount,modulator,modulator amount or volume,solvent,solvent volume,reaction temperature,reaction time
17,"PCN-56, -57, -58 and -59",ZrCl4,24 mg,TPDC derivative dicarboxylic acid ligand,30 or 56 mg,benzoic acid,240 mg,DMF,3.0 mL,100 °C or 120 °C,48 h or 16 h
51,[Zr6O6(OH)2(C10H2O4S3)4(C6H5COO)2(C3H7NO)6](C3...,ZrCl4,223 mg (1 mmol),H2dttdc | Benzoic acid,190 mg (0.67 mmol) | 8 g,N/A,N/A,DMF,50 ml,120 °C,72 hours
52,DUT-52(Zr),ZrCl4,"230 mg, 1.03 mmol","H2(2,6-ndc)","216 mg, 1 mmol",acetic acid,"3 ml, 47.6 mmol",DMF,20 ml,120 °C,24 hours
55,ZrSQU,ZrCl4,0.24 mmol (56 mg),SQA,0.72 mmol (82 mg),HCl,12 mmol (1 ml HCl 37%),"N,N dimethylformamide (DMF) | acetic acid (AA)",5 mL (DMF) | 4 mL (AA),383 K,2-3 h
60,MIL-140A or ZrO[O2C-C6H4-CO2],ZrCl4,20 mmol (4.7 g),"1,4 Benzenedicarboxylic acid (BDC)",40 mmol (6.7 g),N/A,N/A,DMF,"125 mL (118.5g, 1621 mmol)",220 °C,16 hours
...,...,...,...,...,...,...,...,...,...,...,...
322,5,Zn,0.5 mmol,L,"0.18 g, 0.5 mmol",N/A,N/A,ethanol/methanol | DMF | ethanol,10 cm3 | 10 cm3 | 10 cm3,room temperature,2 weeks
17,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
322,6.0,Co,0.5 mmol,L,"0.18 g, 0.5 mmol",N/A,N/A,ethanol/methanol | DMF | ethanol,10 cm3 | 10 cm3 | 10 cm3,room temperature,2 weeks
17,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [ ]:
predict_compound_name = []
predict_metal_source = []
predict_metal_amount = []
predict_linker = []
predict_linker_amount = []
predict_modulator = []
predict_modulator_amount_or_volume = []
predict_solvent = []
predict_solvent_volume = []
predict_reaction_temperature = []
predict_reaction_time = []


ground_truth_compound_name = []
ground_truth_metal_source = []
ground_truth_metal_amount = []
ground_truth_linker = []
ground_truth_linker_amount = []
ground_truth_modulator = []
ground_truth_modulator_amount_or_volume = []
ground_truth_solvent = []
ground_truth_solvent_volume = []
ground_truth_reaction_temperature = []
ground_truth_reaction_time = []

for i,row in df_predict_columns_multiple.iterrows():
    predict_compound_name.append(row['compound name'])
    predict_metal_source.append(row['metal source'])
    predict_metal_amount.append(row['metal amount'])
    predict_linker.append(row['linker'])
    predict_linker_amount.append(row['linker amount'])
    predict_modulator.append(row['modulator'])
    predict_modulator_amount_or_volume.append(row['modulator amount or volume'])
    predict_solvent.append(row['solvent'])
    predict_solvent_volume.append(row['solvent volume'])
    predict_reaction_temperature.append(row['reaction temperature'])
    predict_reaction_time.append(row['reaction time'])

for i,row in df_ground_columns_multiple.iterrows():
    ground_truth_compound_name.append(row['compound name'])
    ground_truth_metal_source.append(row['metal source'])
    ground_truth_metal_amount.append(row['metal amount'])
    ground_truth_linker.append(row['linker'])
    ground_truth_linker_amount.append(row['linker amount'])
    ground_truth_modulator.append(row['modulator'])
    ground_truth_modulator_amount_or_volume.append(row['modulator amount or volume'])
    ground_truth_solvent.append(row['solvent'])
    ground_truth_solvent_volume.append(row['solvent volume'])
    ground_truth_reaction_temperature.append(row['reaction temperature'])
    ground_truth_reaction_time.append(row['reaction time'])

In [ ]:
import joblib
import pandas as pd
import numpy as np
from typing import Tuple, Sequence, List, Optional, Iterable, Any, Iterator, Callable
import textdistance

def levenshtein_similarity(truth: List[str], pred: List[str]) -> float:
    assert len(truth) == len(pred)
    scores = sum(textdistance.levenshtein.normalized_similarity(str(t), str(p)) for t, p in zip(truth, pred))
    return scores / len(truth)


def full_sentence_accuracy(truth: List[str], pred: List[str]) -> float:
    """Calculate the number of exact matches."""
    assert len(truth) == len(pred)
    correct_count = sum(int(t == p) for t, p in zip(truth, pred))
    return correct_count / len(truth)

print("levenshtein_similarity_of_compound_name: ", levenshtein_similarity(ground_truth_compound_name, predict_compound_name))
print("levenshtein_similarity_of_metal_source: ", levenshtein_similarity(ground_truth_metal_source, predict_metal_source))
print("levenshtein_similarity_of_metal_amount: ", levenshtein_similarity(ground_truth_metal_amount, predict_metal_amount))
print("levenshtein_similarity_of_linker: ", levenshtein_similarity(ground_truth_linker, predict_linker))
print("levenshtein_similarity_of_linker_amount: ", levenshtein_similarity(ground_truth_linker_amount, predict_linker_amount))
print("levenshtein_similarity_of_modulator: ", levenshtein_similarity(ground_truth_modulator, predict_modulator))
print("levenshtein_similarity_of_modulator_amount_or_volume: ", levenshtein_similarity(ground_truth_modulator_amount_or_volume, predict_modulator_amount_or_volume))
print("levenshtein_similarity_of_solvent: ", levenshtein_similarity(ground_truth_solvent,predict_solvent))
print("levenshtein_similarity_of_solvent_volume: ", levenshtein_similarity(ground_truth_solvent_volume, predict_solvent_volume))
print("levenshtein_similarity_of_reaction_temperature: ", levenshtein_similarity(ground_truth_reaction_temperature, predict_reaction_temperature))
print("levenshtein_similarity_of_reaction_time: ", levenshtein_similarity(ground_truth_reaction_time, predict_reaction_time))

levenshtein_similarity_mean = (levenshtein_similarity(ground_truth_compound_name, predict_compound_name)
+ levenshtein_similarity(ground_truth_metal_source, predict_metal_source)
+ levenshtein_similarity(ground_truth_metal_amount, predict_metal_amount)
+ levenshtein_similarity(ground_truth_linker, predict_linker)
+ levenshtein_similarity(ground_truth_linker_amount, predict_linker_amount)
+ levenshtein_similarity(ground_truth_modulator, predict_modulator)
+ levenshtein_similarity(ground_truth_modulator_amount_or_volume, predict_modulator_amount_or_volume)
+ levenshtein_similarity(ground_truth_solvent,predict_solvent)
+ levenshtein_similarity(ground_truth_solvent_volume, predict_solvent_volume)
+ levenshtein_similarity(ground_truth_reaction_temperature, predict_reaction_temperature)
+ levenshtein_similarity(ground_truth_reaction_time, predict_reaction_time))/11
print()
print("levenshtein_similarity_mean:",levenshtein_similarity_mean)
print()
print("full_sentence_accuracy_of_compound_name: ", full_sentence_accuracy(ground_truth_compound_name, predict_compound_name))
print("full_sentence_accuracy_of_metal_source: ", full_sentence_accuracy(ground_truth_metal_source, predict_metal_source))
print("full_sentence_accuracy_of_metal_amount: ", full_sentence_accuracy(ground_truth_metal_amount, predict_metal_amount))
print("full_sentence_accuracy_of_linker: ", full_sentence_accuracy(ground_truth_linker, predict_linker))
print("full_sentence_accuracy_of_linker_amount: ", full_sentence_accuracy(ground_truth_linker_amount, predict_linker_amount))
print("full_sentence_accuracy_of_modulator: ", full_sentence_accuracy(ground_truth_modulator, predict_modulator))
print("full_sentence_accuracy_of_modulator_amount_or_volume: ", full_sentence_accuracy(ground_truth_modulator_amount_or_volume, predict_modulator_amount_or_volume))
print("full_sentence_accuracy_of_solvent: ", full_sentence_accuracy(ground_truth_solvent, predict_solvent))
print("full_sentence_accuracy_of_solvent_volume: ", full_sentence_accuracy(ground_truth_solvent_volume, predict_solvent_volume))
print("full_sentence_accuracy_of_reaction_temperature: ", full_sentence_accuracy(ground_truth_reaction_temperature, predict_reaction_temperature))
print("full_sentence_accuracy_of_reaction_time: ", full_sentence_accuracy(ground_truth_reaction_time, predict_reaction_time))

full_sentence_accuracy_mean = (full_sentence_accuracy(ground_truth_compound_name, predict_compound_name)
+ full_sentence_accuracy(ground_truth_metal_source, predict_metal_source)
+ full_sentence_accuracy(ground_truth_metal_amount, predict_metal_amount)
+ full_sentence_accuracy(ground_truth_linker, predict_linker)
+ full_sentence_accuracy(ground_truth_linker_amount, predict_linker_amount)
+ full_sentence_accuracy(ground_truth_modulator, predict_modulator)
+ full_sentence_accuracy(ground_truth_modulator_amount_or_volume, predict_modulator_amount_or_volume)
+ full_sentence_accuracy(ground_truth_solvent,predict_solvent)
+ full_sentence_accuracy(ground_truth_solvent_volume, predict_solvent_volume)
+ full_sentence_accuracy(ground_truth_reaction_temperature, predict_reaction_temperature)
+ full_sentence_accuracy(ground_truth_reaction_time, predict_reaction_time))/11
print()
print("full_sentence_accuracy_mean:",full_sentence_accuracy_mean)


levenshtein_similarity_of_compound_name:  0.7675688596435936
levenshtein_similarity_of_metal_source:  0.7583608452362407
levenshtein_similarity_of_metal_amount:  0.6336395261014767
levenshtein_similarity_of_linker:  0.6839425135604007
levenshtein_similarity_of_linker_amount:  0.5861230582455732
levenshtein_similarity_of_modulator:  0.7300486514203508
levenshtein_similarity_of_modulator_amount_or_volume:  0.6696826135955267
levenshtein_similarity_of_solvent:  0.6047176268211026
levenshtein_similarity_of_solvent_volume:  0.4971682588618072
levenshtein_similarity_of_reaction_temperature:  0.7201665612481551
levenshtein_similarity_of_reaction_time:  0.6998847926267282

levenshtein_similarity_mean: 0.6683003006691778

full_sentence_accuracy_of_compound_name:  0.4731182795698925
full_sentence_accuracy_of_metal_source:  0.6881720430107527
full_sentence_accuracy_of_metal_amount:  0.4731182795698925
full_sentence_accuracy_of_linker:  0.44086021505376344
full_sentence_accuracy_of_linker_amount: 